In [24]:
import os
import pandas as pd
from pathlib import Path

if not "root" in locals():
    current_path = Path(os.getcwd())
    root = current_path.parent.absolute()

In [25]:
os.chdir(root)
from data_treatment.box_creator import create_binary_box

df = pd.read_csv('database.csv')
df = create_binary_box(df, threshold=2., box_length=7).set_index("Date").dropna(axis = 0)
df

,PX_OPEN_VIX_volatility,PX_HIGH_VIX_volatility,PX_LOW_VIX_volatility,PX_LAST_VIX_volatility,VOLUME_TOTAL_CALL_VIX_volatility,PUT_CALL_VOLUME_RATIO_CUR_DAY_VIX_volatility,VOLUME_TOTAL_PUT_VIX_volatility,TOT_OPT_VOLUME_CUR_DAY_VIX_volatility,OPEN_INT_TOTAL_CALL_VIX_volatility,OPEN_INT_TOTAL_PUT_VIX_volatility,...,S&P 500 Weekly Close_SENTIMENT,ST_Rev_F-F_ST_Reversal_Factor_daily,Mkt-RF_F-F_Research_Data_5_Factors_2x3_daily,SMB_F-F_Research_Data_5_Factors_2x3_daily,HML_F-F_Research_Data_5_Factors_2x3_daily,RMW_F-F_Research_Data_5_Factors_2x3_daily,CMA_F-F_Research_Data_5_Factors_2x3_daily,RF_F-F_Research_Data_5_Factors_2x3_daily,Mom _F-F_Momentum_Factor_daily,Box
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-30,14.05,14.18,13.34,13.41,393002.0,0.2628,103300.0,496302.0,5458819.0,1831837.0,...,1875.39,-0.09,0.35,0.22,-0.03,0.08,0.07,0.000,0.53,0
2014-05-02,13.15,13.50,12.83,12.91,376065.0,0.2723,102389.0,478454.0,5917028.0,1919891.0,...,1883.95,0.13,-0.07,0.25,0.43,0.32,0.15,0.000,0.11,0
2014-05-05,13.95,14.20,13.08,13.29,184601.0,0.2980,55014.0,239615.0,5971129.0,1926684.0,...,1883.95,-0.21,0.13,-0.36,-0.55,0.14,-0.27,0.000,0.42,0
2014-05-06,13.65,13.90,13.28,13.80,340114.0,0.1512,51438.0,391552.0,6155777.0,1939953.0,...,1883.95,-0.63,-1.03,-0.50,0.13,0.86,0.12,0.000,-0.44,0
2014-05-07,13.64,14.49,13.39,13.40,278013.0,0.2975,82705.0,360718.0,6296203.0,1963436.0,...,1883.95,-0.56,0.45,-0.74,1.10,0.05,0.53,0.000,-0.21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-13,10.72,10.86,10.33,10.50,530431.0,0.4055,215114.0,745545.0,10032670.0,3249375.0,...,2465.54,0.23,0.11,0.29,0.36,-0.34,0.10,0.005,-0.60,0
2017-09-14,10.57,11.04,10.28,10.44,794561.0,0.4295,341287.0,1135848.0,10399530.0,3281664.0,...,2498.37,-0.20,-0.12,-0.07,-0.07,-0.31,0.24,0.005,0.01,0
2017-09-15,10.51,10.74,10.00,10.17,456381.0,0.4561,208133.0,664514.0,10603850.0,3268040.0,...,2498.37,0.46,0.19,0.26,0.14,0.30,-0.03,0.005,-0.23,0


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

prescale_X = df.drop(columns="Box")
y = df["Box"]
X = (prescale_X - prescale_X.mean())/prescale_X.std()


lsvc = LogisticRegression(penalty="l1", solver="liblinear", C = 0.03).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape

(801, 11)

In [22]:
X.columns[model.get_support()]

Index(['PX_HIGH_VIX_volatility', 'OPEN_INT_TOTAL_CALL_VIX_volatility',
       'VOLATILITY_10D_VIX_volatility', 'VOLATILITY_20D_VIX_volatility',
       'VOLATILITY_10D_SPX_volatility', 'VOLATILITY_20D_SPX_volatility',
       'VOLATILITY_30D_VSTOXX_volatility', 'VOLATILITY_360D_VSTOXX_volatility',
       'BAB Global_BAB Factors', 'Bearish_SENTIMENT',
       'Mkt-RF_F-F_Research_Data_5_Factors_2x3_daily'],
      dtype='object')